In [86]:
1+1

2

In [87]:
from dotenv import load_dotenv

load_dotenv()

True

In [88]:
from langchain_openai import ChatOpenAI
import fitz  # pymupdf
import os
import faiss
import numpy as np
from langchain_openai import OpenAIEmbeddings
import pickle
from datetime import datetime

llm = ChatOpenAI(model="gpt-4o", streaming=True)

In [89]:
# LangGraph 관련 라이브러리
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode  # ToolExecutor 대신 사용
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from typing import TypedDict, List, Dict, Any
import json

In [90]:
# def extract_text_with_fitz(pdf_path):
#     """PyMuPDF로 텍스트 추출 - 한글 처리 개선"""
#     try:
#         doc = fitz.open(pdf_path)
#         print(f"PDF 파일: {pdf_path}")
#         print(f"총 페이지 수: {len(doc)}")
        
#         full_text = ""
        
#         for page_num in range(len(doc)):
#             print(f"페이지 {page_num + 1} 처리 중...")
#             page = doc[page_num]
            
#             # 텍스트 추출
#             text = page.get_text()
            
#             if text and text.strip():
#                 # 한글 처리 및 텍스트 정리
#                 text = text.replace('\x00', '')  # null 문자 제거
#                 text = text.replace('\ufeff', '')  # BOM 제거
#                 text = text.replace('\r\n', '\n')  # 줄바꿈 정리
#                 text = text.replace('\r', '\n')
                
#                 print(f"  페이지 {page_num + 1}: {len(text)} 글자 추출")
#                 print(f"  첫 50글자: {text[:50]}")
                
#                 full_text += text + "\n\n"  # 페이지 구분
#             else:
#                 print(f"  페이지 {page_num + 1}: 텍스트 없음")
        
#         doc.close()
#         print(f"\n총 추출된 텍스트: {len(full_text)} 글자")
#         return full_text
        
#     except Exception as e:
#         print(f"PyMuPDF 오류: {e}")
#         return ""

def extract_text_with_fitz(pdf_path):
    """PyMuPDF로 텍스트 추출 - 한글 처리 개선"""
    try:
        doc = fitz.open(pdf_path)
        print(f"PDF 파일: {pdf_path}")
        print(f"총 페이지 수: {len(doc)}")
        
        full_text = ""
        
        for page_num in range(len(doc)):
            print(f"페이지 {page_num + 1} 처리 중...")
            page = doc[page_num]
            
            # 텍스트 추출
            text = page.get_text()
            
            if text and text.strip():
                # 한글 처리 및 텍스트 정리
                text = text.replace('\x00', '')  # null 문자 제거
                text = text.replace('\ufeff', '')  # BOM 제거
                text = text.replace('\r\n', '\n')  # 줄바꿈 정리
                text = text.replace('\r', '\n')
                
                print(f"  페이지 {page_num + 1}: {len(text)} 글자 추출")
                print(f"  첫 50글자: {text[:50]}")
                
                full_text += text + "\n\n"  # 페이지 구분
            else:
                print(f"  페이지 {page_num + 1}: 텍스트 없음")
        
        doc.close()
        print(f"\n총 추출된 텍스트: {len(full_text)} 글자")
        return full_text
        
    except Exception as e:
        print(f"PyMuPDF 오류: {e}")
        return ""

# 텍스트 추출
pdf_file = "2024-PR-15.pdf"
full_text = extract_text_with_fitz(pdf_file)

if full_text:
    print("✓ 텍스트 추출 성공!")
    print(f"전체 텍스트 첫 200글자:")
    print(full_text[:300])
else:
    print("✗ 텍스트 추출 실패")

PDF 파일: 2024-PR-15.pdf
총 페이지 수: 127
페이지 1 처리 중...
  페이지 1: 61 글자 추출
  첫 50글자: 약자와 동행하는 서울의 교통
이신해   김승준   한영준   양재환 
윤서연   연준형  
페이지 2 처리 중...
  페이지 2: 16 글자 추출
  첫 50글자: 약자와 동행하는 서울의 교통

페이지 3 처리 중...
  페이지 3: 257 글자 추출
  첫 50글자: 이 보고서의 내용은 연구진의 견해로서 
서울특별시의 정책과는 다를 수도 있습니다.
연구책임
페이지 4 처리 중...
  페이지 4: 1010 글자 추출
  첫 50글자: i
약
자
와
 동
행
하
는
 서
울
의
 교
통
서울시, 다양한 교통약자 불편 개선 위
페이지 5 처리 중...
  페이지 5: 979 글자 추출
  첫 50글자: ii
요
약
서울 장시간 통근자 지원 시급…혼잡 완화·서비스 수준 보장 방안 필요
교통 체
페이지 6 처리 중...
  페이지 6: 321 글자 추출
  첫 50글자: iii
약
자
와
 동
행
하
는
 서
울
의
 교
통
목차
01 연구개요
2
1_연구배경
페이지 7 처리 중...
  페이지 7: 649 글자 추출
  첫 50글자: iv
목
차
표목차
[표 2-1] 교통부문에서의 약자 정의와 권리, 책무 
14
[표 3-
페이지 8 처리 중...
  페이지 8: 752 글자 추출
  첫 50글자: v
약
자
와
 동
행
하
는
 서
울
의
 교
통
그림목차
[그림 1-1] 약자의 범위 
페이지 9 처리 중...
  페이지 9: 1125 글자 추출
  첫 50글자: vi
목
차
[그림 3-15] 고령자가 걸어서 동네 외출 시 가장 불편한 점
38
[그림 
페이지 10 처리 중...
  페이지 10: 820 글자 추출
  첫 50글자: vii
약
자
와
 동
행
하
는
 서
울
의
 교
통
[그림 4-24] 현재 거주지 선택
페이지 11 처리 중...
  페이지 11: 32 글자 추출
  첫 50글자: 01
연구개요
1_연구

In [91]:
# def chunk_text(text, chunk_size=2000, overlap=100):
#     """텍스트를 청크로 나누는 함수"""
#     if not text.strip():
#         print("빈 텍스트입니다.")
#         return []
    
#     chunks = []
#     start = 0
    
#     while start < len(text):
#         end = start + chunk_size
#         chunk = text[start:end].strip()
        
#         if chunk:  # 빈 청크가 아닌 경우만 추가
#             chunks.append(chunk)
        
#         start = end - overlap
    
#     return chunks

def chunk_text(text, chunk_size=2000, overlap=100):
    """텍스트를 청크로 나누는 함수"""
    if not text.strip():
        print("빈 텍스트입니다.")
        return []
    
    chunks = []
    start = 0
    
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end].strip()
        
        if chunk:  # 빈 청크가 아닌 경우만 추가
            chunks.append(chunk)
        
        start = end - overlap
    
    return chunks

# 텍스트 청킹
if full_text:
    chunks = chunk_text(full_text)
    
    print(f"\n=== 청킹 결과 ===")
    print(f"총 청크 개수: {len(chunks)}")
    
    if chunks:
        chunk_lengths = [len(chunk) for chunk in chunks]
        print(f"청크 길이 - 최소: {min(chunk_lengths)}, 최대: {max(chunk_lengths)}, 평균: {sum(chunk_lengths)/len(chunk_lengths):.1f}")
        
        print(f"\n첫 번째 청크 미리보기:")
        print(chunks[0][:200] + "..." if len(chunks[0]) > 200 else chunks[0])
        
        print(f"\n마지막 청크 미리보기:")
        print(chunks[-1][:200] + "..." if len(chunks[-1]) > 200 else chunks[-1])


=== 청킹 결과 ===
총 청크 개수: 49
청크 길이 - 최소: 129, 최대: 2000, 평균: 1961.4

첫 번째 청크 미리보기:
약자와 동행하는 서울의 교통
이신해   김승준   한영준   양재환 
윤서연   연준형   김영범   김지한


약자와 동행하는 서울의 교통


이 보고서의 내용은 연구진의 견해로서 
서울특별시의 정책과는 다를 수도 있습니다.
연구책임
이신해 서울연구원 스마트교통연구실 선임연구위원
연구진
김승준 서울연구원 스마트교통연구실 선임연구위원
한영준 서울연구원 스마...

마지막 청크 미리보기:
-15
발행인  오균
발행일  2025년 5월 30일
발행처  서울연구원
ISBN  979-11-5700-908-4 95350  비매품
06756 서울특별시 서초구 남부순환로 340길 57
이 출판물의 판권은 서울연구원에 속합니다.


In [92]:
def search_documents_openai(query, index, embedding_model, chunks, metadatas, k=3):
    """
    OpenAI 임베딩을 사용한 문서 검색 (누락된 함수)
    """
    try:
        # 쿼리를 임베딩으로 변환
        query_embedding = embedding_model.embed_query(query)
        query_embedding = np.array([query_embedding]).astype('float32')
        
        # FAISS로 유사한 문서 검색
        distances, indices = index.search(query_embedding, k)
        
        # 결과 포맷팅
        results = []
        for i, (distance, idx) in enumerate(zip(distances[0], indices[0])):
            if idx < len(chunks):  # 유효한 인덱스인지 확인
                results.append({
                    'chunk_id': idx,
                    'content': chunks[idx],
                    'score': 1.0 - (distance / 2.0),  # 거리를 유사도로 변환
                    'metadata': metadatas[idx] if idx < len(metadatas) else {}
                })
        
        return results
        
    except Exception as e:
        print(f"검색 오류: {e}")
        return []

In [93]:
import faiss
import numpy as np
from langchain_openai import OpenAIEmbeddings
import pickle
import os
from datetime import datetime

# OpenAI API 키 설정 (환경변수에서 읽어오거나 직접 설정)
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"  # 필요한 경우

def save_vectorstore_openai(index, chunks, metadatas, pdf_filename, save_name="pdf_openai_vectors"):
    """벡터 스토어 저장"""
    try:
        # FAISS 인덱스 저장
        faiss.write_index(index, f'{save_name}_vectors.index')
        
        # 청크와 메타데이터 저장
        with open(f'{save_name}_data.pkl', 'wb') as f:
            pickle.dump({
                'chunks': chunks,
                'metadatas': metadatas,
                'pdf_filename': pdf_filename,
                'chunk_count': len(chunks),
                'embedding_model': 'text-embedding-3-large',
                'embedding_type': 'openai'
            }, f)
        
        return True
        
    except Exception as e:
        print(f"저장 오류: {e}")
        return False

def create_faiss_vectorstore_openai(chunks, pdf_filename, save_name="pdf_openai_vectors"):
    """청크들을 FAISS 벡터 스토어로 변환하고 자동 저장 (OpenAI 임베딩 사용)"""
    if not chunks:
        print("청크가 없습니다.")
        return None, None, None, None
    
    print("OpenAI 임베딩 모델 로드 중...")
    embedding = OpenAIEmbeddings(model='text-embedding-3-large')
    
    print(f"임베딩 생성 중... ({len(chunks)}개 청크)")
    print("OpenAI API 호출 중이므로 시간이 걸릴 수 있습니다...")
    
    try:
        # 청크들을 임베딩으로 변환
        embeddings = embedding.embed_documents(chunks)
        embeddings = np.array(embeddings).astype('float32')
        
        print(f"임베딩 완료! 차원: {embeddings.shape[1]}, 개수: {embeddings.shape[0]}")
        
        # FAISS 인덱스 생성
        dimension = embeddings.shape[1]
        index = faiss.IndexFlatL2(dimension)  # L2 거리 기반 인덱스
        index.add(embeddings)
        
        # 메타데이터 생성
        metadatas = []
        for i, chunk in enumerate(chunks):
            metadatas.append({
                'chunk_id': i,
                'source': pdf_filename,
                'chunk_size': len(chunk),
                'preview': chunk[:100] + "..." if len(chunk) > 100 else chunk
            })
        
        print(f"FAISS 벡터 스토어 생성 완료! {index.ntotal}개 벡터 저장됨")
        
        # 자동 저장
        print("벡터 스토어를 자동 저장 중...")
        if save_vectorstore_openai(index, chunks, metadatas, pdf_filename, save_name):
            print("✓ 자동 저장 완료!")
        else:
            print("⚠️  자동 저장 실패")
        
        return index, embedding, chunks, metadatas
        
    except Exception as e:
        print(f"임베딩 생성 오류: {e}")
        print("OpenAI API 키가 설정되어 있는지 확인해주세요.")
        return None, None, None, None

def load_or_create_vectorstore(chunks, pdf_filename, save_name="pdf_openai_vectors"):
    """
    벡터 스토어가 있으면 로드하고, 없으면 새로 생성하는 함수
    """
    index_file = f'{save_name}_vectors.index'
    data_file = f'{save_name}_data.pkl'
    
    # 저장된 파일들이 모두 존재하는지 확인
    if os.path.exists(index_file) and os.path.exists(data_file):
        print("기존 벡터 스토어를 찾았습니다. 로드 중...")
        
        try:
            # 저장된 데이터 로드
            index = faiss.read_index(index_file)
            
            with open(data_file, 'rb') as f:
                data = pickle.load(f)
            
            # 저장된 PDF 파일명과 현재 파일명 비교
            if data['pdf_filename'] == pdf_filename:
                print("✓ 동일한 PDF 파일의 벡터 스토어 로드 성공!")
                
                # OpenAI 임베딩 모델 로드
                embedding_model = OpenAIEmbeddings(model='text-embedding-3-large')
                
                print(f"- 벡터 개수: {index.ntotal}")
                print(f"- 청크 개수: {data['chunk_count']}")
                print(f"- 원본 파일: {data['pdf_filename']}")
                print(f"- 임베딩 모델: {data.get('embedding_model', '정보 없음')}")
                
                return index, embedding_model, data['chunks'], data['metadatas']
            else:
                print(f"⚠️  다른 PDF 파일의 벡터 스토어입니다.")
                print(f"   저장된 파일: {data['pdf_filename']}")
                print(f"   현재 파일: {pdf_filename}")
                print("   새로운 벡터 스토어를 생성합니다...")
                
        except Exception as e:
            print(f"⚠️  벡터 스토어 로드 중 오류 발생: {e}")
            print("   새로운 벡터 스토어를 생성합니다...")
    
    else:
        print("저장된 벡터 스토어를 찾을 수 없습니다. 새로 생성합니다...")
    
    # # 새로운 벡터 스토어 생성
    # print("\n=== 새로운 벡터 스토어 생성 ===")
    # return create_faiss_vectorstore_openai(chunks, pdf_filename, save_name)

# FAISS 벡터 스토어 자동 로드/생성
if 'chunks' in locals() and chunks:
    print("=== 벡터 스토어 처리 ===")
    index, embedding_model, chunks, metadatas = load_or_create_vectorstore(
        chunks, 
        pdf_file, 
        save_name="my_pdf_vectors"  # 원하는 저장 이름으로 변경 가능
    )
    
    if index is not None:
        print("✓ 벡터 스토어 준비 완료!")
        print("이제 검색을 할 수 있습니다!")
    else:
        print("✗ 벡터 스토어 준비 실패")
else:
    print("청크가 없습니다. 먼저 텍스트 추출과 청킹을 실행해주세요.")

=== 벡터 스토어 처리 ===
기존 벡터 스토어를 찾았습니다. 로드 중...
✓ 동일한 PDF 파일의 벡터 스토어 로드 성공!
- 벡터 개수: 49
- 청크 개수: 49
- 원본 파일: 2024-PR-15.pdf
- 임베딩 모델: text-embedding-3-large
✓ 벡터 스토어 준비 완료!
이제 검색을 할 수 있습니다!


In [94]:
# 필요한 라이브러리들
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolExecutor
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from typing import TypedDict, List, Dict, Any
import json

# 이미 가지고 있는 검색 관련 함수들 활용

In [95]:
class AgentState(TypedDict):
    messages: List[Any]
    user_query: str
    search_results: List[Dict]
    summary: str
    tool_calls: List[Dict]

In [96]:
@tool
def search_pdf_documents(query: str, k: int = 3) -> str:
    """
    PDF 문서에서 관련 내용을 검색합니다.
    
    Args:
        query: 검색할 키워드나 질문
        k: 반환할 결과 개수 (기본값: 3)
    
    Returns:
        검색된 문서 내용들
    """
    global index, embedding_model, chunks, metadatas
    
    if 'index' not in globals() or index is None:
        return "벡터 스토어가 준비되지 않았습니다. 먼저 PDF를 처리해주세요."
    
    try:
        # 검색 실행
        results = search_documents_openai(query, index, embedding_model, chunks, metadatas, k)
        
        if not results:
            return f"'{query}'에 대한 검색 결과가 없습니다."
        
        # 검색 결과를 문자열로 포맷팅
        formatted_results = []
        for i, result in enumerate(results):
            formatted_results.append(
                f"[검색결과 {i+1}]\n"
                f"유사도: {result['score']:.4f}\n"
                f"내용: {result['content'][:500]}{'...' if len(result['content']) > 500 else ''}\n"
            )
        
        return "\n\n".join(formatted_results)
        
    except Exception as e:
        return f"검색 중 오류 발생: {str(e)}"

In [97]:
@tool
def summarize_content(content: str, focus: str = "general") -> str:
    """
    주어진 내용을 요약합니다.
    
    Args:
        content: 요약할 내용
        focus: 요약 초점 ("general", "key_points", "technical", "brief")
    
    Returns:
        요약된 내용
    """
    if not content.strip():
        return "요약할 내용이 없습니다."
    
    # OpenAI LLM 모델 사용
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
    
    focus_prompts = {
        "general": "다음 내용을 전반적으로 요약해주세요:",
        "key_points": "다음 내용에서 핵심 포인트들만 정리해주세요:",
        "technical": "다음 내용에서 기술적인 부분을 중심으로 요약해주세요:",
        "brief": "다음 내용을 간단히 한 문단으로 요약해주세요:"
    }
    
    prompt = focus_prompts.get(focus, focus_prompts["general"])
    
    try:
        response = llm.invoke([
            HumanMessage(content=f"{prompt}\n\n{content}")
        ])
        return response.content
    except Exception as e:
        return f"요약 중 오류 발생: {str(e)}"

In [98]:
@tool
def get_document_info() -> str:
    """
    현재 로드된 PDF 문서의 정보를 반환합니다.
    
    Returns:
        문서 정보
    """
    global chunks, metadatas, pdf_file
    
    if 'chunks' not in globals() or not chunks:
        return "로드된 문서가 없습니다."
    
    total_chars = sum(len(chunk) for chunk in chunks)
    
    return f"""
현재 로드된 문서 정보:
- 파일명: {pdf_file if 'pdf_file' in globals() else '알 수 없음'}
- 총 청크 수: {len(chunks)}
- 총 글자 수: {total_chars:,}
- 평균 청크 크기: {total_chars // len(chunks):,} 글자
"""

In [99]:
def should_continue(state: AgentState) -> str:
    """다음 단계를 결정하는 함수"""
    messages = state["messages"]
    last_message = messages[-1]
    
    # 도구 호출이 있으면 도구 실행
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"
    
    # 그렇지 않으면 종료
    return END

def call_model(state: AgentState) -> AgentState:
    """LLM 모델 호출"""
    messages = state["messages"]
    
    # 시스템 프롬프트
    system_prompt = """
당신은 PDF 문서 검색 및 요약 전문 에이전트입니다.

사용 가능한 도구들:
1. search_pdf_documents: PDF에서 관련 내용 검색
2. summarize_content: 내용 요약
3. get_document_info: 문서 정보 조회

사용자의 질문에 따라 적절한 도구를 사용하여 답변하세요:
- 특정 내용을 찾고 싶다면 search_pdf_documents를 사용
- 검색된 내용을 요약하고 싶다면 summarize_content를 사용
- 문서 정보가 궁금하다면 get_document_info를 사용

항상 한국어로 친절하고 정확하게 답변해주세요.
"""
    
    # 시스템 메시지가 없다면 추가
    if not messages or not any(isinstance(msg, HumanMessage) for msg in messages[:1]):
        messages = [HumanMessage(content=system_prompt)] + messages
    
    # LLM 호출
    llm = ChatOpenAI(model="gpt-4o", temperature=0.1)
    llm_with_tools = llm.bind_tools([search_pdf_documents, summarize_content, get_document_info])
    
    response = llm_with_tools.invoke(messages)
    
    return {"messages": messages + [response]}

# 도구 노드 생성 (ToolExecutor 대신 ToolNode 사용)
tools = [search_pdf_documents, summarize_content, get_document_info]
tool_node = ToolNode(tools)

def call_tools(state: AgentState) -> AgentState:
    """도구 실행 - ToolNode를 사용한 새로운 방식"""
    return tool_node.invoke(state)

# def call_tools(state: AgentState) -> AgentState:
#     """도구 실행"""
#     messages = state["messages"]
#     last_message = messages[-1]
    
#     # 도구 실행기 생성
#     tools = [search_pdf_documents, summarize_content, get_document_info]
#     tool_executor = ToolExecutor(tools)
    
#     tool_messages = []
    
#     # 각 도구 호출 실행
#     for tool_call in last_message.tool_calls:
#         try:
#             result = tool_executor.invoke(tool_call)
#             tool_messages.append(
#                 ToolMessage(
#                     content=str(result),
#                     tool_call_id=tool_call["id"]
#                 )
#             )
#         except Exception as e:
#             tool_messages.append(
#                 ToolMessage(
#                     content=f"도구 실행 오류: {str(e)}",
#                     tool_call_id=tool_call["id"]
#                 )
#             )
    
#     return {"messages": messages + tool_messages}

In [100]:
def create_pdf_agent():
    """PDF 검색 요약 에이전트 생성"""
    
    # 그래프 생성
    graph_builder = StateGraph(AgentState)

    # 노드 추가
    graph_builder.add_node("agent", call_model)
    graph_builder.add_node("tools", call_tools)

    # 시작점 설정
    graph_builder.set_entry_point("agent")

    # 조건부 엣지 추가
    graph_builder.add_conditional_edges(
        "agent",
        should_continue,
        {
            "tools": "tools",
            END: END
        }
)

# 도구에서 다시 에이전트로
    graph_builder.add_edge("tools", "agent")

# 그래프 컴파일
    graph = graph_builder.compile()

    graph

# 에이전트 생성
# pdf_agent = create_pdf_agent()


In [101]:
def run_pdf_agent(user_query: str, pdf_agent) -> str:
    """PDF 에이전트 실행"""
    
    # 초기 상태
    initial_state = {
        "messages": [HumanMessage(content=user_query)],
        "user_query": user_query,
        "search_results": [],
        "summary": "",
        "tool_calls": []
    }
    
    try:
        # 에이전트 실행
        result = pdf_agent.invoke(initial_state)
        
        # 최종 응답 추출
        final_message = result["messages"][-1]
        if hasattr(final_message, 'content'):
            return final_message.content
        else:
            return str(final_message)
            
    except Exception as e:
        return f"에이전트 실행 중 오류 발생: {str(e)}"

def interactive_pdf_agent(pdf_agent):
    """대화형 PDF 에이전트"""
    print("\n" + "="*60)
    print("🤖 PDF 검색 요약 에이전트")
    print("="*60)
    print("• '종료', 'quit', 'exit'를 입력하면 종료됩니다")
    print("• 예시 질문:")
    print("  - '머신러닝에 대해 검색해서 요약해줘'")
    print("  - '데이터 분석 관련 내용을 찾아줘'")
    print("  - '문서 정보를 알려줘'")
    print("-"*60)
    
    while True:
        try:
            user_input = input("\n💬 질문을 입력하세요: ").strip()
            
            if user_input.lower() in ['종료', 'quit', 'exit', 'q']:
                print("에이전트를 종료합니다. 👋")
                break
            
            if not user_input:
                continue
            
            print("\n🤖 에이전트가 작업 중입니다...")
            print("-" * 40)
            
            # 에이전트 실행
            response = run_pdf_agent(user_input, pdf_agent)
            
            print(f"\n🎯 답변:\n{response}")
            print("-" * 60)
            
        except KeyboardInterrupt:
            print("\n\n에이전트를 종료합니다. 👋")
            break
        except Exception as e:
            print(f"오류 발생: {e}")

In [102]:
# 벡터 스토어가 준비되었는지 확인
if 'index' in locals() and index is not None:
    print("✅ PDF 검색 요약 에이전트가 준비되었습니다!")
    
    # 테스트 실행
    test_query = "문서 정보를 알려줘"
    print(f"\n🧪 테스트 실행: {test_query}")
    result = run_pdf_agent(test_query)
    print(f"결과: {result}")
    
    # 대화형 실행
    print("\n🚀 대화형 에이전트를 시작하시겠습니까? (y/n)")
    if input("입력: ").lower() in ['y', 'yes', '네', 'ㅇ']:
        interactive_pdf_agent()
        
else:
    print("❌ 벡터 스토어가 준비되지 않았습니다.")
    print("먼저 PDF 처리와 벡터 스토어 생성을 완료해주세요.")

✅ PDF 검색 요약 에이전트가 준비되었습니다!

🧪 테스트 실행: 문서 정보를 알려줘


TypeError: run_pdf_agent() missing 1 required positional argument: 'pdf_agent'

In [ ]:
import gradio as gr

# 위의 모든 코드가 실행된 후에 사용
def create_gradio_interface():
    """Gradio 인터페이스 생성"""
    
    def chat_with_agent(message, history):
        """Gradio용 채팅 함수"""
        try:
            # 전역 변수 확인
            if 'pdf_agent' not in globals() or pdf_agent is None:
                return history + [[message, "에이전트가 초기화되지 않았습니다. 먼저 PDF 처리를 완료해주세요."]], ""
            
            # 에이전트 실행
            response = run_pdf_agent(message, pdf_agent)
            
            # 히스토리에 추가
            history.append([message, response])
            return history, ""
            
        except Exception as e:
            error_msg = f"오류가 발생했습니다: {str(e)}"
            history.append([message, error_msg])
            return history, ""
    
    # Gradio 인터페이스
    with gr.Blocks(title="🤖 PDF 문서 AI 어시스턴트", theme="soft") as demo:
        
        gr.Markdown("""
        # 🤖 PDF 문서 AI 어시스턴트
        
        **기능:**
        - 📄 PDF 문서 내용 검색
        - 📝 검색 결과 요약
        - 💬 자연어 질의응답
        
        **예시 질문:**
        - "교통약자에 대해 검색해서 요약해줘"
        - "고령자 정책은 어떤 것들이 있어?"
        - "문서 정보를 알려줘"
        """)
        
        chatbot = gr.Chatbot(
            value=[["시스템", "안녕하세요! PDF 문서에 대해 질문해보세요."]],
            height=500,
            show_label=False
        )
        
        with gr.Row():
            msg_input = gr.Textbox(
                placeholder="질문을 입력하세요...",
                show_label=False,
                scale=4
            )
            send_btn = gr.Button("전송", variant="primary", scale=1)
        
        # 예시 버튼들
        with gr.Row():
            gr.Button("📄 문서 정보"),
            gr.Button("🔍 교통약자 검색"),
            gr.Button("📝 고령자 정책 요약"),
            gr.Button("🚌 대중교통 내용")
        
        # 이벤트 연결
        msg_input.submit(chat_with_agent, [msg_input, chatbot], [chatbot, msg_input])
        send_btn.click(chat_with_agent, [msg_input, chatbot], [chatbot, msg_input])
    
    return demo

# 실행
if 'pdf_agent' in globals() and pdf_agent is not None:
    print("🚀 Gradio 인터페이스 준비 완료!")
    demo = create_gradio_interface()
    demo.launch(share=False, server_port=7864, debug=True)

In [ ]:
if __name__ == "__main__":
    # 1. PDF 텍스트 추출
    pdf_file = "2024-PR-15.pdf"
    if os.path.exists(pdf_file):
        full_text = extract_text_with_fitz(pdf_file)
        
        if full_text:
            print("✓ 텍스트 추출 성공!")
            
            # 2. 텍스트 청킹
            chunks = chunk_text(full_text)
            print(f"✓ 청킹 완료: {len(chunks)}개 청크")
            
            # 3. 벡터 스토어 생성/로드
            index, embedding_model, chunks, metadatas = load_or_create_vectorstore(
                chunks, 
                pdf_file, 
                save_name="my_pdf_vectors"
            )
            
            if index is not None:
                print("✓ 벡터 스토어 준비 완료!")
                
                # 4. 에이전트 생성
                pdf_agent = create_pdf_agent()
                print("✓ PDF 에이전트 생성 완료!")
                
                # 5. 테스트 실행
                test_query = "문서 정보를 알려줘"
                print(f"\n🧪 테스트 실행: {test_query}")
                result = run_pdf_agent(test_query, pdf_agent)
                print(f"결과: {result}")
                
                # 6. 대화형 실행
                print("\n🚀 대화형 에이전트를 시작하시겠습니까? (y/n)")
                if input("입력: ").lower() in ['y', 'yes', '네', 'ㅇ']:
                    interactive_pdf_agent(pdf_agent)
                    
            else:
                print("✗ 벡터 스토어 준비 실패")
        else:
            print("✗ 텍스트 추출 실패")
    else:
        print(f"✗ PDF 파일을 찾을 수 없습니다: {pdf_file}")

In [ ]:
# %pip install streamlit

In [ ]:
# import gradio as gr
# from langchain_core.messages import HumanMessage, AIMessage
# from langchain_openai import ChatOpenAI


# def answer_invoke(message, history):
#     try:
#         # 1. LangGraph 에이전트 사용 (기존 rag_chain 대신)
#         # 초기 상태 설정
#         initial_state = {
#             "messages": [HumanMessage(content=message)],
#             "user_query": message,
#             "search_results": [],
#             "summary": "",
#             "tool_calls": []
#         }
        
#         # 2. 에이전트 실행
#         result = graph.invoke(initial_state)  # pdf_agent 대신 graph 사용
        
#         # 3. 최종 응답 추출
#         final_message = result["messages"][-1]
#         if hasattr(final_message, 'content'):
#             return final_message.content
#         else:
#             return str(final_message)
            
#     except Exception as e:
#         return f"죄송합니다. 오류가 발생했습니다: {str(e)}"

# # Gradio 인터페이스 설정
# demo = gr.ChatInterface(
#     fn=answer_invoke,
#     title="📚 요약봇 (LangGraph)",
#     description="LangGraph 에이전트를 사용한 PDF 문서 기반 질답 시스템",
#     examples=[
#         "이 문서 전체를 요약해줘",
#         "결론 부분만 요약해줘",
#         "고령자 관련 내용을 요약해줘",
#         "서울의 교통 정책에 대해 요약해서 알려줘"
#     ],
#     theme="soft"
# )

# # 실행
# if __name__ == "__main__":
#     demo.launch(
#         share=True,      # 공개 링크 생성
#         server_name="0.0.0.0",  # 외부 접속 허용
#         server_port=7860         # 포트 설정
#     )